In [1]:
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-06-01 06:59:59.483255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 07:00:01.982302: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
ncol = 384
nlev = 60
tlen = 10091520

In [3]:
# Convert to pressure
def convert_to_Pressure(hyai,hybi,PS,P0):
    Dimension1=hyai.shape
    Dimension2=PS.shape
    Pressure = np.zeros([Dimension2[1],Dimension1[0]])
    
    for i in range(Dimension2[1]):
        Pressure[i,:] = (P0*hyai[:] + PS[0,i]*hybi[:])
    return Pressure

In [4]:
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
lon = ne4_grid_info.lon.values
lat = ne4_grid_info.lat.values
area = ne4_grid_info.area.values
hyai = ne4_grid_info.hyai.values
hybi = ne4_grid_info.hybi.values
PS = ne4_grid_info.PS.values
P0 = ne4_grid_info.P0.values

In [5]:
Pre = convert_to_Pressure(hyai,hybi,PS,P0)

In [6]:
dPre = Pre[:,1:nlev+1] - Pre[:,0:nlev] 

In [7]:
mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')

def load_nc_dir_with_generator_test(filelist:list, latlim=[-999,999], lonlim=[-999,999]):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            ds = ds.merge(ne4_grid_info[['lat','lon']])
            ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
            ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            dso = dso.merge(ne4_grid_info[['lat','lon']])
            dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
            dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

In [8]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [9]:
# validation dataset for HPO
stride_sample = 5 
f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0009-01-*-*.nc')
f_mli_val = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli = f_mli_val
print(f'#files: {len(f_mli)}')

#files: 26280


In [10]:
tds_test = load_nc_dir_with_generator_test(f_mli)
work = list(tds_test.as_numpy_iterator())
x_true_n = np.concatenate([ work[k][0] for k in range(len(work)) ])
y_true_n = np.concatenate([ work[k][1] for k in range(len(work)) ])

2023-06-01 07:00:32.781141: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-01 07:00:32.933399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [11]:
# This function turns 20 minute increments into daily averages
# (384)*ndays*72 = Dim[0]
# Output dim: (384)*ndays
# Griffin's original version is based on regrided data
# This version is based on ne4pg2 grid (384 columns)
# This change allow me to use Sungduk's code directly
# since he generaged the min/max/scale already
def average_data_Griffin_LiranEdited(reconstructed_targets,reconstructed_features):
    Dim = reconstructed_targets.shape
    x = 384
    tnum = 72
    t = Dim[0]
    z = Dim[1]
    ndays = int(t/(tnum*x))
    
    target_days = np.zeros(shape=(x*ndays,tnum, z))
    feat_days = np.zeros(shape=(x*ndays,tnum, z))
    day_array_targ = np.zeros(shape=(x,tnum,ndays, z))
    day_array_feat = np.zeros(shape=(x,tnum,ndays, z))
    #print(day_array_feat.shape)
    ncol_count = 0
    tstep_count = 0
    day_count = 0
    
    for i in range(t):
        temp_targ = reconstructed_targets[i, :]
        day_array_targ[ncol_count,tstep_count,day_count, :] = temp_targ
        temp_feat = reconstructed_features[i, :]
        day_array_feat[ncol_count,tstep_count,day_count,:] = temp_feat
        
        if (ncol_count == x-1):
            ncol_count = 0
            tstep_count = tstep_count+1
        else:
            ncol_count = ncol_count+1
        
        if (tstep_count == tnum):
            tstep_count = 0
            day_count = day_count+1   
            
            
    day_array_targ_out = np.nanmean(day_array_targ, axis = 1)
    day_array_feat_out = np.nanmean(day_array_feat, axis = 1)
    
    return day_array_targ_out,day_array_feat_out

In [12]:
import os
import netCDF4 as nc
folder_path = "/pscratch/sd/h/heroplr/step2_retrain/backup_phase-4_retrained_models"  # Replace with the actual folder path
out_folder = "/pscratch/sd/h/heroplr/R2_analysis_all/"
Dim_true = x_true_n.shape
# Loop through all files in the folder
numday = int(Dim_true[0]/ncol/72)
filename="step2_lot-101_trial_0028.best.h5"
file_path = os.path.join(folder_path, filename)

In [13]:
model = keras.models.load_model(file_path,compile=False)
y_pred = model(x_true_n)
T_tend_true = y_true_n[:,:60]
T_pred_true = y_pred[:,:60]
Q_tend_true = y_true_n[:,60:120]
Q_pred_true =y_pred[:,60:120]

In [89]:
mweightpre = np.zeros([ncol,nlev])
for k in range(nlev):
    mweightpre[:,k]    =  area*dPre[:,k]/np.sum(dPre[:,k])/9.8 # area*dP/g

In [98]:
mweightpre.shape

(384, 60)

In [25]:
T_tend_true2,T_pred_true2 = average_data_Griffin_LiranEdited(T_tend_true,T_pred_true)
Q_tend_true2,Q_pred_true2 = average_data_Griffin_LiranEdited(Q_tend_true,Q_pred_true)

In [102]:
cp_dry_o   = 1006
latent_heat_vaporization = 2.501e6 
T_tend_true3 = np.zeros([ncol,365,nlev])
T_pred_true3 = np.zeros([ncol,365,nlev])
Q_tend_true3 = np.zeros([ncol,365,nlev])
Q_pred_true3 = np.zeros([ncol,365,nlev])
for k in range(365):
    T_tend_true3[:,k,:] = T_tend_true2[:,k,:]*mweightpre*cp_dry_o
    T_pred_true3[:,k,:] = T_pred_true2[:,k,:]*mweightpre*cp_dry_o
    Q_tend_true3[:,k,:] = Q_tend_true2[:,k,:]*mweightpre*latent_heat_vaporization
    Q_pred_true3[:,k,:] = Q_pred_true2[:,k,:]*mweightpre*latent_heat_vaporization



In [103]:
T_tend_true4 = np.reshape(T_tend_true3, (384, 365*60))
T_pred_true4 = np.reshape(T_pred_true3, (384, 365*60))
Q_tend_true4 = np.reshape(Q_tend_true3, (384, 365*60))
Q_pred_true4 = np.reshape(Q_pred_true3, (384, 365*60))

In [104]:
Tdiff = T_pred_true4-T_tend_true4
Tdiff2 = Tdiff**2
Tdiff2_rmse = np.sqrt(Tdiff2)

In [106]:
Qdiff2_rmse.shape

(384, 21900)

In [105]:
Qdiff = Q_pred_true4-Q_tend_true4
Qdiff2 = Qdiff**2
Qdiff2_rmse = np.sqrt(Qdiff2)

In [110]:
Tdiff2_rmsemean = np.mean(Tdiff2_rmse,axis=1)
Qdiff2_rmsemean = np.mean(Qdiff2_rmse,axis=1)

In [111]:
np.sum(Qdiff2_rmsemean)

64.65253187890707

In [112]:
np.sum(Tdiff2_rmsemean)

0.018307799332078198

In [81]:
# Create a new NetCDF file
filename = file_path[-31:]+"_mwv20.nc"
file_path_out = os.path.join(out_folder, filename)
        
ncfile = nc.Dataset(file_path_out, "w", format="NETCDF4")

# Define the dimensions
time_dim = ncfile.createDimension("time", None)  # Unlimited dimension
lat_dim = ncfile.createDimension("ncol", ncol)
lon_dim = ncfile.createDimension("nlev", nlev)
# Create variables
time_var = ncfile.createVariable("time", "f8", ("time",))
lon_var = ncfile.createVariable("lon", "f4", ("ncol",))
lat_var = ncfile.createVariable("lat", "f4", ("ncol",))
data_var3 = ncfile.createVariable("T_tend_true2", "f8", ("nlev","time","ncol"))
data_var4 = ncfile.createVariable("T_pred_true2", "f8", ("nlev","time","ncol"))
data_var5 = ncfile.createVariable("Q_tend_true2", "f8", ("nlev","time","ncol"))
data_var6 = ncfile.createVariable("Q_pred_true2", "f8", ("nlev","time","ncol"))

data_var7 = ncfile.createVariable("Trmsemean", "f8", ("ncol"))
data_var8 = ncfile.createVariable("Qrmsemean", "f8", ("ncol"))

# Assign values to variables
time_var[:] = [365]  # Example time values
lon_var[:] = lon    # Example latitude values
lat_var[:] = lat  # Example longitude values

data_var3[:,:] = np.transpose(T_tend_true2)
data_var4[:,:] = np.transpose(T_pred_true2)
data_var5[:,:] = np.transpose(Q_tend_true2)
data_var6[:,:] = np.transpose(Q_pred_true2)
data_var7[:] = (Tdiff2_rmsemean)
data_var8[:] = (Qdiff2_rmsemean)
# Add global attributes
ncfile.description = "R2"
ncfile.history = "Created by Liran"

# Close the NetCDF file
ncfile.close()
    

In [82]:
file_path_out

'/pscratch/sd/h/heroplr/R2_analysis_all/tep2_lot-101_trial_0028.best.h5_mwv20.nc'